In [1]:
using MacroTools
using BenchmarkTools 
using TupleTools
using Test
using ReactiveMP
using GraphPPL

import MacroTools: prewalk, postwalk

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1423
┌ Warning: Package ReactiveMP does not have Unrolled in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Unrolled as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Unrolled into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.
┌ Info: Precompiling GraphPPL [b3f8163a-e979-4e85-b43e-1f63d8c8b42c]
└ @ Base loading.jl:1423


In [2]:
macro e(something)
    if something.head === :macrocall
        return esc(:(println(MacroTools.prettify(@macroexpand $something)); $something))
    end
    return esc(something)
end

@e (macro with 1 method)

In [3]:
cs = @constraints function test()
    q(x, y) = q(x)q(y)
    q(x, y, t, r) = q(x, y)q(t)q(r)
    q(x, w) = q(x)q(w)
    q(y, w) = q(y)q(w)
    q(x) = q(x[begin:begin+2])q(x[begin+3])..q(x[end])
end
        
@time test()
@time test()

cs = test()

  0.000010 seconds (14 allocations: 2.062 KiB)
  0.000010 seconds (14 allocations: 2.062 KiB)


Constraints:
	form: ()
	factorisation
		q(x, y) = q(x)q(y)
		q(x, y, t, r) = q(x, y)q(t)q(r)
		q(x, w) = q(x)q(w)
		q(y, w) = q(y)q(w)
		q(x) = q(x[(begin):((begin) + 2)])q(x[((begin) + 3)..(end)])


In [4]:
model = Model()

x = randomvar(model, :x, 10)
y = randomvar(model, :y, 10)
tmp = randomvar(model, :tmp)
t = randomvar(model, :t, 10)
r = randomvar(model, :r)

vars = (x[3], x[4], y[1], t[1]);
# vars = (xvar, yvar, tvar)

In [5]:
@time ReactiveMP.resolve_factorisation(:(1 + 1), vars, cs, model)
@time ReactiveMP.resolve_factorisation(:(1 + 1), vars, cs, model)

  1.103759 seconds (2.67 M allocations: 146.779 MiB, 2.65% gc time, 99.92% compilation time)
  0.000064 seconds (187 allocations: 6.609 KiB)


((1,), (2,), (3,), (4,))

In [6]:
@time ReactiveMP.resolve_factorisation(:(1 + 1), vars, cs, model)
@time ReactiveMP.resolve_factorisation(:(1 + 1), vars, cs, model)

  0.000068 seconds (187 allocations: 6.609 KiB)
  0.000085 seconds (187 allocations: 6.609 KiB)


((1,), (2,), (3,), (4,))

In [7]:
@btime ReactiveMP.resolve_factorisation(:(1 + 1), $vars, $cs, $model)

  6.924 μs (181 allocations: 6.42 KiB)


((1,), (2,), (3,), (4,))

In [8]:
# fast playground
let 
    cs = @constraints begin
        q(x, y) = q(x[begin], y[begin])..q(x[end], y[end])
        # q(x, y) = (q(x[begin])..q(x[end]))*(q(y[begin])..q(y[end]))        
        q(x, y, t) = q(x, y)q(t)
        q(x, y, r) = q(x, y)q(r)
    end
            
    model = Model()
            
    y = randomvar(model, :y, 10)
    x = randomvar(model, :x, 10)
    t = randomvar(model, :t)
    r = randomvar(model, :r)
    
    vars = (y[1], y[2], x[1], x[2], t, r)
    
    @time ReactiveMP.resolve_factorisation(:(1 + 1), vars, cs, model)
    @time ReactiveMP.resolve_factorisation(:(1 + 1), vars, cs, model)
    
    @btime ReactiveMP.resolve_factorisation(:(1 + 1), $vars, $cs, $model)
end

  0.666946 seconds (1.65 M allocations: 89.508 MiB, 3.93% gc time, 99.90% compilation time)
  0.000064 seconds (331 allocations: 13.547 KiB)
  13.108 μs (325 allocations: 13.36 KiB)


((1, 3), (2, 4), (5, 6))

In [9]:
# fast playground
let 
    cs = @constraints begin
        q(x, y) = (q(x[begin])..q(x[end]))*(q(y[begin])..q(y[end]))        
        q(x, y, t) = q(x, y)q(t)
        q(x, y, r) = q(x, y)q(r)
    end
            
    model = Model()
            
    y = randomvar(model, :y, 10)
    x = randomvar(model, :x, 10)
    t = randomvar(model, :t)
    r = randomvar(model, :r)
    
    vars = (y[1], y[2], x[1], x[2], t, r)
    
    @time ReactiveMP.resolve_factorisation(:(1 + 1), vars, cs, model)
    @time ReactiveMP.resolve_factorisation(:(1 + 1), vars, cs, model)
    
    @btime ReactiveMP.resolve_factorisation(:(1 + 1), $vars, $cs, $model)
end

  0.215115 seconds (607.40 k allocations: 32.503 MiB, 99.86% compilation time)
  0.000061 seconds (307 allocations: 12.562 KiB)
  11.256 μs (301 allocations: 12.38 KiB)


((1,), (2,), (3,), (4,), (5, 6))

In [10]:
# fast playground
let 
    cs = @constraints begin
        q(x, y) = q(y[1])q(x[begin], y[begin+1])..q(x[end], y[end])       
        q(x, y, t) = q(x, y)q(t)
        q(x, y, r) = q(x, y)q(r)
    end
            
    model = Model()
            
    y = randomvar(model, :y, 11)
    x = randomvar(model, :x, 10)
    t = randomvar(model, :t)
    r = randomvar(model, :r)
    
    vars = (y[1], y[2], x[1], x[2], t, r)
    
    @time ReactiveMP.resolve_factorisation(:(1 + 1), vars, cs, model)
    @time ReactiveMP.resolve_factorisation(:(1 + 1), vars, cs, model)
    
    @btime ReactiveMP.resolve_factorisation(:(1 + 1), $vars, $cs, $model)
end

  0.418719 seconds (1.10 M allocations: 59.179 MiB, 3.14% gc time, 99.91% compilation time)
  0.000064 seconds (395 allocations: 15.141 KiB)
  15.326 μs (389 allocations: 14.95 KiB)


((1,), (2, 3), (4,), (5, 6))

In [11]:
# fast playground
let 
    cs = @constraints begin
        q(x, y) = q(x)q(y)
    end
            
    model = Model()
            
    y = randomvar(model, :y, 11)
    x = randomvar(model, :x, 10)
    
    vars = (y[1], y[2], x[1], x[2])
    
    @time ReactiveMP.resolve_factorisation(:(1 + 1), vars, cs, model)
    @time ReactiveMP.resolve_factorisation(:(1 + 1), vars, cs, model)
    
    @btime ReactiveMP.resolve_factorisation(:(1 + 1), $vars, $cs, $model)
end

  0.100430 seconds (300.36 k allocations: 15.434 MiB, 99.91% compilation time)
  0.000025 seconds (89 allocations: 3.750 KiB)
  3.713 μs (83 allocations: 3.56 KiB)


((1, 2), (3, 4))

In [12]:
# fast playground
let 
    cs = @constraints begin
        q(x, y) = q(x)q(y)
    end
            
    model = Model()
            
    y = randomvar(model, :y, 11)
    x = randomvar(model, :x, 10)
    
    vars = (y[1], x[1])
    
    @time ReactiveMP.resolve_factorisation(:(1 + 1), vars, cs, model)
    @time ReactiveMP.resolve_factorisation(:(1 + 1), vars, cs, model)
    
    @btime ReactiveMP.resolve_factorisation(:(1 + 1), $vars, $cs, $model)
end

  0.182169 seconds (449.99 k allocations: 23.848 MiB, 6.74% gc time, 99.89% compilation time)
  0.000021 seconds (49 allocations: 1.844 KiB)
  2.049 μs (43 allocations: 1.66 KiB)


((1,), (2,))

In [13]:
# fast playground
let 
    @constraints function withflag(flag)
        if flag
            q(x, y) = q(x)q(y)
        else
            q(x, y) = q(x, y)
        end
    end
            
    model = Model()
            
    y = randomvar(model, :y, 11)
    x = randomvar(model, :x, 10)
    
    vars = (y[1], x[1])
    
    @show ReactiveMP.resolve_factorisation(:(1 + 1), vars, withflag(true), model)
    @show ReactiveMP.resolve_factorisation(:(1 + 1), vars, withflag(false), model)
end

ReactiveMP.resolve_factorisation($(Expr(:quote, :(1 + 1))), vars, withflag(true), model) = ((1,), (2,))
ReactiveMP.resolve_factorisation($(Expr(:quote, :(1 + 1))), vars, withflag(false), model) = ((1, 2),)


((1, 2),)

In [14]:
# fast playground
let 
    @constraints function withflag(n)
        q(x, y) = q(x[1:n])*(q(x[n+1])..q(x[end]))*q(y)
    end
            
    model = Model()
            
    y = randomvar(model, :y, 11)
    x = randomvar(model, :x, 10)
    
    vars = (y[1], x[2], x[3])
    
    @show ReactiveMP.resolve_factorisation(:(1 + 1), vars, withflag(2), model)
    @show ReactiveMP.resolve_factorisation(:(1 + 1), vars, withflag(4), model)
end

ReactiveMP.resolve_factorisation($(Expr(:quote, :(1 + 1))), vars, withflag(2), model) = ((1,), (2,), (3,))
ReactiveMP.resolve_factorisation($(Expr(:quote, :(1 + 1))), vars, withflag(4), model) = ((1,), (2, 3))


((1,), (2, 3))

In [15]:
# fast playground
let 
    cs = @constraints begin
       q(x, y) = q(x[begin])*q(x[begin+1:end])*q(y)
    end
            
    model = Model()
            
    y = randomvar(model, :y, 11)
    x = randomvar(model, :x, 10)
    
    @show ReactiveMP.resolve_factorisation(:(1 + 1), (y[1], x[1], x[2]), cs, model)
    @show ReactiveMP.resolve_factorisation(:(1 + 1), (y[1], x[2], x[3]), cs, model)
end

ReactiveMP.resolve_factorisation($(Expr(:quote, :(1 + 1))), (y[1], x[1], x[2]), cs, model) = ((1,), (2,), (3,))
ReactiveMP.resolve_factorisation($(Expr(:quote, :(1 + 1))), (y[1], x[2], x[3]), cs, model) = ((1,), (2, 3))


((1,), (2, 3))

In [16]:
# Should error
@constraints begin
    q(x, y) = q(x[begin], x[begin])..q(x[end], x[end])q(y)
end

LoadError: Cannot split q(x[(begin)], x[(begin)]) and q(x[(end)], x[(end)]). Names should be unique.